In [143]:
from icecube import icetray, dataio, dataclasses, simclasses, phys_services
import os, sys
import numpy as np
import matplotlib as mpl
from matplotlib.colors import LogNorm, Normalize
import matplotlib.pyplot as plt
import timeit as time
import math
from datetime import datetime

In [144]:
plt.rcParams.update({'font.size': 20})
mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Bitstream Vera Sans'
mpl.rcParams['mathtext.it'] = 'Bitstream Vera Sans:italic'
mpl.rcParams['mathtext.bf'] = 'Bitstream Vera Sans:bold'

mpl.rcParams['hist.bins'] = 100
mpl.rcParams['lines.linewidth'] = 2.5
mpl.rcParams['patch.linewidth'] = 2.5
mpl.rcParams['grid.alpha'] = 0.2

In [145]:
frame_arr = []; mctree = []
track_list = []

for i in range(83, 84):
    files_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/' + str(i) + '.i3.bz2' # path to i3 file
    with dataio.I3File('/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/'+ str(i) + '.i3.bz2') as infile:
        for frame in infile:
            if infile.stream.id != 'P': continue
            frame_arr.append(frame)
            mctree.append(frame['I3MCTree']) 
            

In [146]:
#mctree_arr = np.array(mctree_arr)
#mmc_Ei = []
#for frame in frame_arr:
#    for i in range(len(frame['MMCTrackList'])):
#        mmc_tr = frame['MMCTrackList'][i]
#        mmc_Ei.append(mmc_tr.Ei/icetray.I3Units.GeV)
#
#mmc_Ei = np.array(mmc_Ei)
#print(mmc_Ei)

In [147]:
#print(mctree)
muons_pos = []
muons_length = []
entering_large_cylinder = []
d_injection_cylinder = []
d_inside = []
e_ion = []
daughters = []


for frame in frame_arr:
    for i in range(len(frame['MMCTrackList'])):
        mmc_tr = frame['MMCTrackList'][i]
        mctree = frame['I3MCTree']
        
        muons_pos.append(mmc_tr.particle.pos)
        muons_length.append(mmc_tr.particle.length)
        entering_large_cylinder.append(dataclasses.I3Position(mmc_tr.xi, mmc_tr.yi, mmc_tr.zi))
        d_injection_cylinder.append((mmc_tr.particle.pos - dataclasses.I3Position(mmc_tr.xi, mmc_tr.yi, mmc_tr.zi)).magnitude)
        d_inside.append(mmc_tr.particle.length - (mmc_tr.particle.pos - dataclasses.I3Position(mmc_tr.xi, mmc_tr.yi, mmc_tr.zi)).magnitude)
        e_ion.append((1/4.5)*(mmc_tr.particle.length - (mmc_tr.particle.pos - dataclasses.I3Position(mmc_tr.xi, mmc_tr.yi, mmc_tr.zi)).magnitude))
        daughters=(mctree.get_daughters(mmc_tr.particle))
        
        
        
            
muons_pos = np.array(muons_pos)
muons_lenght = np.array(muons_length)
entering_large_cylinder = np.array(entering_large_cylinder)
d_injection_cylinder = np.array(d_injection_cylinder)
e_ion = np.array(e_ion)
daughters = np.array(daughters)



print('Injection point of the first muon in file 83', muons_pos[0])
print('Length that the first muon in file 83 traveled in total', muons_length[0])
print('Point where it enters the 800m volume', entering_large_cylinder[0])
print('Distance from injection to 800m volume', d_injection_cylinder[0])
print('Ionization losses', e_ion[0])
#print(daughters)

Injection point of the first muon in file 83 [-227.48417893 1017.21534929 1949.99477481]
Length that the first muon in file 83 traveled in total 3125.5943701684264
Point where it enters the 800m volume [-115.17625567  542.97290035  800.        ]
Distance from injection to 800m volume 1249.0023827390578
Ionization losses 417.0204416509708


In [148]:


for frame in frame_arr: 
    for i in range(len(frame['MMCTrackList'])):
        mmc_tr = frame['MMCTrackList'][i]                              
        mctree = frame['I3MCTree']
        daughters = mctree.get_daughters(mmc_tr.particle)


        e_stochastic = 0
        for d in daughters:
            
            e_stochastic += d.energy
print('Stochastic losses',e_stochastic)



Stochastic losses 35.82414376918573


In [257]:
muon_track = []
print(np.shape(track_list))

for i in range(len(track_list)):
    muon_track.append(track_list[i])
    print([i],'Injection point of the muon', muon_track[i].particle.pos)
    print('Length that the muon traveled in total', muon_track[i].particle.length)
    # Let's calculate the distance the muon traveled *inside the 800m volume*
    entering_large_cylinder = dataclasses.I3Position(muon_track[i].xi, muon_track[i].yi, muon_track[i].zi)
    print('Point where it enters the 800m volume', entering_large_cylinder)
    d_injection_cylinder = (muon_track[i].particle.pos - entering_large_cylinder).magnitude
    print('Distance from injection to 800m volume', d_injection_cylinder)
    d_inside = muon_track[i].particle.length - d_injection_cylinder

    # Ionization energy loss
    econstant = 1/4.5 # GeV/m
    e_ion = econstant*d_inside
    print('Ionization losses', e_ion)

    # Sum all the stochastics
    daughters = mctree.get_daughters(muon_track[i].particle)
    e_stochastic = 0
    for d in daughters:
        e_stochastic += d.energy

    print('Stochastic losses', e_stochastic)
    print('Total E lost inside volume', e_stochastic+e_ion)
    print('Energy available upon entering volume', muon_track[i].Ei)
    
    print('Entrance point of the particle (x,y,z)', dataclasses.I3Position(muon_track[i].xi, muon_track[i].yi, muon_track[i].zi))
    print('Length that the muon traveled in 800 (Length)', d_inside)

    daughters = mctree.get_daughters(muon_track[i].particle)


    e_stochastic = 0
    for d in daughters:
        position=d.pos
        if np.sqrt((position.x)**2 + (position.y)**2) <= 500.0 and  position.z>=-500.0 and position.z <= 500.0:
            e_stochastic += d.energy
    print('Stochastic losses in 500m',e_stochastic)


(2631,)


AttributeError: 'I3MMCTrackList' object has no attribute 'particle'